# A Pleasent Flight
 Historical Air Crash + Delay Investigation

Unfortunely, there were many aviation accidents that caused innocent people to perish. While lessons we learnt in the past tragedy have made flying now more and more safer, the figures of planes falling from the sky still frightens a few people from having a pleasent flight. In this data analysis, we are going persuade thoes who are afraid by to investigate how safe are planes and, if one is still concerned, how to maximize one's safety? To make our report also interesting to general public, we will also investigate how one can minimize delays. 

In [93]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
import matplotlib.pyplot as plt
import requests

import xml.etree.ElementTree as ET # to read one dataset in XML format

## Clean Air Crash Data

In [94]:
crash_df = pd.read_csv("datasets/crash/Airplane_Crashes_and_Fatalities_Since_1908.csv")
etree = ET.parse("datasets/crash/AviationData.xml")

load data

select business flights whose enginetype is not reciprocating

In [95]:
xml_root = etree.getroot()

interest_columns = ["EventId","EventDate","Location","Country","Latitude","Longitude","AirportCode","InjurySeverity","AircraftDamage",
                    "AircraftCategory","NumberOfEngines","EngineType","Schedule","TotalUninjured","TotalMinorInjuries",
                    "TotalSeriousInjuries","TotalFatalInjuries","WeatherCondition","BroadPhaseOfFlight","RegistrationNumber","PurposeOfFlight"]

NTSB_crash_df = pd.DataFrame(columns=interest_columns) # NTSB_crash_df: dataframe which collects airport information. -- YD 02/28/2020

In [96]:
for elem in xml_root: # This loop will run only once
    for row in elem: 
        if not(row.attrib["PurposeOfFlight"]=="Business"):  # Only care about business flights whose engine type is not 'Reciprocating'-- YD 02/28/2020
            continue
        if row.attrib["EngineType"]=="Reciprocating": 
            continue
        information = list()
        for interest in interest_columns:
            if not (row.attrib[interest]==""):
                information.append(row.attrib[interest])
            else:
                information.append(np.nan)
        row_information = pd.Series(information,index=interest_columns)
        NTSB_crash_df = NTSB_crash_df.append(row_information,ignore_index=True)
            
# this may need to run for a while. It takes 18 seconds on my computer
NTSB_crash_df

,EventId,EventDate,Location,Country,Latitude,Longitude,AirportCode,InjurySeverity,AircraftDamage,AircraftCategory,...,EngineType,Schedule,TotalUninjured,TotalMinorInjuries,TotalSeriousInjuries,TotalFatalInjuries,WeatherCondition,BroadPhaseOfFlight,RegistrationNumber,PurposeOfFlight
0,20200128X02229,01/22/2020,"Minot, ND",United States,48.232223,-101.293333,2ND4,Non-Fatal,Substantial,Helicopter,...,Turbo Shaft,NaN,1,NaN,NaN,NaN,VMC,LANDING,N947LH,Business
1,20191031X31506,10/31/2019,"New Braunfels, TX",United States,29.705833,-98.043333,BAZ,Non-Fatal,Minor,Airplane,...,NaN,NaN,1,NaN,1,NaN,VMC,NaN,N850BK,Business
2,20191019X04853,10/19/2019,"Albuquerque, NM",United States,35.084444,-106.650278,NaN,Non-Fatal,Substantial,Balloon,...,None,NaN,13,NaN,NaN,NaN,VMC,LANDING,N275WB,Business
3,20191021X62154,10/16/2019,"Augusta, GA",United States,33.371945,-81.971667,AGS,Non-Fatal,NaN,Airplane,...,Turbo Fan,NaN,NaN,NaN,1,NaN,VMC,TAXI,N307KP,Business
4,20191005X82852,10/05/2019,"Kokomo, IN",United States,40.475000,-86.063333,OKK,Fatal(1),Destroyed,Airplane,...,NaN,NaN,NaN,NaN,NaN,1,VMC,MANEUVERING,N326CW,Business
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,20020917X02196,03/12/1982,"CHINO, CA",United States,NaN,NaN,CNO,Non-Fatal,Substantial,Airplane,...,Turbo Jet,NaN,2,0,0,0,VMC,LANDING,N479CC,Business
831,20020917X01930,03/01/1982,"ALMA, CO",United States,NaN,NaN,NaN,Non-Fatal,Substantial,Helicopter,...,Turbo Shaft,NaN,2,0,0,0,VMC,CRUISE,N580P,Business
832,20020917X01727,02/16/1982,"SPRINGFIELD, KY",United States,NaN,NaN,6I2,Fatal(8),Destroyed,Airplane,...,Turbo Prop,NaN,0,0,0,8,IMC,APPROACH,N2517X,Business
833,20020917X01923,02/03/1982,"HAYDEN, CO",United States,NaN,NaN,HDN,Non-Fatal,Destroyed,Airplane,...,Turbo Prop,NaN,0,6,0,0,IMC,LANDING,N149JA,Business


select all events with valid airport code

In [97]:
cities_need_df = pd.read_csv('cities_need.csv')
# cities_need_df

In [98]:
NTSB_crash_df['CheckAirportCode'] = NTSB_crash_df['AirportCode'].isin(cities_need_df['AirportCode'])
NTSB_crash_df = NTSB_crash_df[NTSB_crash_df['CheckAirportCode'] == True]
NTSB_crash_df

,EventId,EventDate,Location,Country,Latitude,Longitude,AirportCode,InjurySeverity,AircraftDamage,AircraftCategory,...,Schedule,TotalUninjured,TotalMinorInjuries,TotalSeriousInjuries,TotalFatalInjuries,WeatherCondition,BroadPhaseOfFlight,RegistrationNumber,PurposeOfFlight,CheckAirportCode
3,20191021X62154,10/16/2019,"Augusta, GA",United States,33.371945,-81.971667,AGS,Non-Fatal,NaN,Airplane,...,NaN,NaN,NaN,1,NaN,VMC,TAXI,N307KP,Business,True
5,20191003X13056,10/03/2019,"Lansing, MI",United States,42.780278,-84.606111,LAN,Fatal(5),Substantial,Airplane,...,NaN,NaN,NaN,1,5,IMC,APPROACH,N700AQ,Business,True
26,20181130X80724,11/30/2018,"Fargo, ND",United States,46.920556,-96.815833,FAR,Non-Fatal,Substantial,Airplane,...,NaN,2,9,NaN,NaN,IMC,APPROACH,N941JM,Business,True
57,20170228X14333,02/27/2017,"Bellingham, WA",United States,48.791944,-122.534722,BLI,Non-Fatal,Substantial,Airplane,...,NaN,1,NaN,NaN,NaN,IMC,TAKEOFF,CGWVS,Business,True
58,20170222X75136,02/20/2017,"Albuquerque, NM",United States,35.064722,-106.628056,ABQ,Non-Fatal,Substantial,Balloon,...,NaN,10,NaN,NaN,NaN,VMC,APPROACH,N709TC,Business,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,20001214X42006,01/20/1983,"LUBBOCK, TX",United States,NaN,NaN,LBB,Non-Fatal,Substantial,NaN,...,NaN,2,0,0,0,IMC,LANDING,N555CH,Business,True
806,20020917X04375,12/14/1982,"COLUMBIA, SC",United States,NaN,NaN,CAE,Non-Fatal,Substantial,Airplane,...,NaN,2,0,0,0,VMC,LANDING,N522MA,Business,True
812,20020917X04497,09/12/1982,"HAYDEN, CO",United States,NaN,NaN,HDN,Fatal(6),Destroyed,Airplane,...,NaN,0,0,0,6,VMC,TAKEOFF,N66U,Business,True
818,20020917X03513,08/16/1982,"EL PASO, TX",United States,NaN,NaN,ELP,Incident,Minor,Airplane,...,NaN,0,1,0,0,VMC,DESCENT,N1969J,Business,True


delete unrelated columns

In [99]:
NTSB_crash_df = NTSB_crash_df.drop(columns = ['EventId', 'EventDate', 'Location', 'Country', 'Latitude', 'Longitude', 'Schedule', 'RegistrationNumber', 'PurposeOfFlight', 'CheckAirportCode'])
NTSB_crash_df = NTSB_crash_df.reset_index(drop = True)
NTSB_crash_df

,AirportCode,InjurySeverity,AircraftDamage,AircraftCategory,NumberOfEngines,EngineType,TotalUninjured,TotalMinorInjuries,TotalSeriousInjuries,TotalFatalInjuries,WeatherCondition,BroadPhaseOfFlight
0,AGS,Non-Fatal,NaN,Airplane,2,Turbo Fan,NaN,NaN,1,NaN,VMC,TAXI
1,LAN,Fatal(5),Substantial,Airplane,1,Turbo Prop,NaN,NaN,1,5,IMC,APPROACH
2,FAR,Non-Fatal,Substantial,Airplane,2,Turbo Jet,2,9,NaN,NaN,IMC,APPROACH
3,BLI,Non-Fatal,Substantial,Airplane,1,Turbo Prop,1,NaN,NaN,NaN,IMC,TAKEOFF
4,ABQ,Non-Fatal,Substantial,Balloon,NaN,NaN,10,NaN,NaN,NaN,VMC,APPROACH
...,...,...,...,...,...,...,...,...,...,...,...,...
104,LBB,Non-Fatal,Substantial,NaN,2,Turbo Prop,2,0,0,0,IMC,LANDING
105,CAE,Non-Fatal,Substantial,Airplane,2,Turbo Prop,2,0,0,0,VMC,LANDING
106,HDN,Fatal(6),Destroyed,Airplane,2,Turbo Prop,0,0,0,6,VMC,TAKEOFF
107,ELP,Incident,Minor,Airplane,2,Turbo Prop,0,1,0,0,VMC,DESCENT


Standardize data

In [103]:
# This function deletes appended number behind data, blank spaces, parenthesis and lower case all letters -- YD 02/28/2020
# e.g. Fatal(10) --> fatal
def standarize_DeleteAppendedNumber(data_str):
    out=""
    for i in range(len(data_str)):
        if data_str[i] == " " or data_str[i] == "(" or data_str[i] == ")":
            continue
        elif data_str[i].isnumeric():
            continue
        else:
            out += data_str[i].lower()
    return out

In [104]:
NTSB_crash_df['InjurySeverity'] = NTSB_crash_df['InjurySeverity'].apply(standarize_DeleteAppendedNumber)

In [105]:
# NTSB_crash_df['InjurySeverity'].value_counts()

non-fatal    74
incident     18
fatal        17
Name: InjurySeverity, dtype: int64

EDA